In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
import nltk
from nltk.stem import PorterStemmer , WordNetLemmatizer
import re
from nltk.corpus import wordnet,stopwords
from sklearn.metrics.pairwise import sigmoid_kernel , cosine_similarity
import string

In [8]:
df = pd.read_csv('/content/drive/MyDrive/datasets/book_data.csv')

In [9]:
df.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54301 entries, 0 to 54300
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   book_authors       54301 non-null  object 
 1   book_desc          52970 non-null  object 
 2   book_edition       5453 non-null   object 
 3   book_format        52645 non-null  object 
 4   book_isbn          41435 non-null  object 
 5   book_pages         51779 non-null  object 
 6   book_rating        54301 non-null  float64
 7   book_rating_count  54301 non-null  int64  
 8   book_review_count  54301 non-null  int64  
 9   book_title         54301 non-null  object 
 10  genres             51059 non-null  object 
 11  image_url          53618 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 5.0+ MB


In [6]:
data = df.copy()

In [7]:
data.drop(['image_url','book_format','book_edition','book_isbn'],axis=1,inplace=True)

In [8]:
data.book_authors = data.book_authors.apply(lambda x : " ".join([i for i in x.split('|')]))

In [10]:
data['book_desc'] = data['book_desc'].fillna('')
data['genres'] = data['genres'].fillna('')

In [11]:
data.isnull().sum()

book_authors            0
book_desc               0
book_pages           2522
book_rating             0
book_rating_count       0
book_review_count       0
book_title              0
genres                  0
dtype: int64

In [12]:
data.genres = data.genres.apply(lambda x : " ".join([i for i in x.split('|')]))

In [13]:
data.head()

,book_authors,book_desc,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres
0,Suzanne Collins,Winning will make you famous. Losing means cer...,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult Fiction Science Fiction Dystopia F...
1,J.K. Rowling Mary GrandPré,There is a door at the end of a silent corrido...,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy Young Adult Fiction
2,Harper Lee,The unforgettable novel of a childhood in a sl...,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics Fiction Historical Historical Fiction...
3,Jane Austen Anna Quindlen Mrs. Oliphant George...,«È cosa ormai risaputa che a uno scapolo in po...,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics Fiction Romance
4,Stephenie Meyer,About three things I was absolutely positive.F...,498 pages,3.58,4281268,97991,Twilight,Young Adult Fantasy Romance Paranormal Vampire...


In [14]:
data['text_data'] = data['book_authors']  + " " + data['book_desc'] + data['genres'] 

In [11]:
new_data = data.drop(['book_authors','book_authors','book_desc','genres'],axis=1)

NameError: ignored

In [12]:
new_data.head()

,Unnamed: 0,book_pages,book_rating,book_rating_count,book_review_count,book_title,text_data,clean_data
0,0,374 pages,4.33,5519135,160706,The Hunger Games,Suzanne Collins Winning will make you famous. ...,uzanne collins win make famous lose mean certa...
1,1,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,J.K. Rowling Mary GrandPré There is a door at ...,k rowling mary grandpré door end silent corrid...
2,2,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Harper Lee The unforgettable novel of a childh...,arper lee unforgettable novel childhood sleepy...
3,3,279 pages,4.25,2453620,54322,Pride and Prejudice,Jane Austen Anna Quindlen Mrs. Oliphant George...,ane austen anna quindlen mrs oliphant george s...
4,4,498 pages,3.58,4281268,97991,Twilight,Stephenie Meyer About three things I was absol...,tephenie meyer three thing absolutely positive...


In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
def get_wordnet_pos(word):

    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN) 

In [16]:
lemma = WordNetLemmatizer()
stopword = stopwords.words('english')
ps = PorterStemmer()


In [17]:
def clean_text(i):
    i = i.lower()
    i = re.sub(r'^[a-z0-9]','',i)
    i = " ".join([lemma.lemmatize(i,get_wordnet_pos(i)) for i in i.split() if i not in stopword])
    return i 

In [24]:
new_data['clean_data'] = new_data['text_data'].apply(lambda x : clean_text(x))

In [27]:
new_data.clean_data = new_data.clean_data.apply(lambda x : "".join([i for i in x if i not in string.punctuation]) )

In [5]:
new_data.to_csv('book_data_clean_text.csv')

NameError: ignored

In [3]:
new_data = pd.read_csv('/content/drive/MyDrive/datasets/book_data_clean_text.csv')

In [4]:
new_data.head()

,Unnamed: 0,book_pages,book_rating,book_rating_count,book_review_count,book_title,text_data,clean_data
0,0,374 pages,4.33,5519135,160706,The Hunger Games,Suzanne Collins Winning will make you famous. ...,uzanne collins win make famous lose mean certa...
1,1,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,J.K. Rowling Mary GrandPré There is a door at ...,k rowling mary grandpré door end silent corrid...
2,2,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Harper Lee The unforgettable novel of a childh...,arper lee unforgettable novel childhood sleepy...
3,3,279 pages,4.25,2453620,54322,Pride and Prejudice,Jane Austen Anna Quindlen Mrs. Oliphant George...,ane austen anna quindlen mrs oliphant george s...
4,4,498 pages,3.58,4281268,97991,Twilight,Stephenie Meyer About three things I was absol...,tephenie meyer three thing absolutely positive...


In [5]:
new_data = new_data.drop_duplicates(subset='book_title', keep="first")

In [6]:
new_data['book_pages'] = new_data['book_pages'].fillna('500 pages')

In [7]:
new_data['book_pages'] = new_data.book_pages.apply(lambda x : int(x.split()[0]))

In [8]:
new_data_1 = new_data[ (new_data.book_rating > 4.0) & (new_data.book_pages > 200) ]

In [9]:
new_data_1.to_csv('book_data_rating_pages.csv')

In [10]:
new_data = pd.read_csv('/content/book_data_rating_pages.csv')

In [11]:
new_data_1.shape

(19948, 8)

In [12]:
corpus = list(new_data.clean_data)

In [13]:

dets = new_data[['book_title','clean_data']]

In [14]:
tfidf = TfidfVectorizer()
Cv = CountVectorizer()
ncorpus = tfidf.fit_transform(corpus)

In [15]:
cosine_similarities = cosine_similarity(ncorpus)


In [16]:
names = dets.book_title.tolist()
names[:5]

['The Hunger Games',
 'Harry Potter and the Order of the Phoenix',
 'To Kill a Mockingbird',
 'Pride and Prejudice',
 'The Book Thief']

In [ ]:
# values = []
# for i in cosine_similarities:
#   val = []
#   for n , j in enumerate(i) :
#     val.append((n,j))
#   values.append(val)

In [17]:
def get_recom_cosin(title,func=cosine_similarities):
  idx = names.index('The Hunger Games')
  similar = cosine_similarities[idx]
  values= []
  for n,i in enumerate(similar):
    values.append((n,i))
  #values= list(enumerate(similar))
  scores = sorted(values,key = lambda x : x[1],reverse=True)
  scores = scores[1:11]
  idxs = [i[0] for i in scores]
  return  new_data.iloc[idxs]

In [18]:
new_data.iloc[40]

Unnamed: 0                                                          70
Unnamed: 0.1                                                        70
book_pages                                                         850
book_rating                                                       4.22
book_rating_count                                               668422
book_review_count                                                40168
book_title                                                   Outlander
text_data            Diana Gabaldon The year is 1945. Claire Randal...
clean_data           iana gabaldon year 1945 claire randall former ...
Name: 40, dtype: object

In [19]:
name = input("Enter book title : ")
data = get_recom_cosin(name)
data[['book_pages','book_title','book_rating']]


Enter book title : The Stand


,book_pages,book_title,book_rating
143,391,Catching Fire,4.29
203,392,Mockingjay,4.03
12169,424,Catching Fire - Tersulut,4.29
12266,400,L'Embrasement,4.29
9301,339,De Hongerspelen,4.33
17062,407,En llamas,4.29
10158,351,Vlammen,4.29
9364,369,Hunger Games,4.33
107,1155,The Hunger Games Trilogy Boxset,4.48
3721,304,Escape from Mr. Lemoncello's Library,4.13


In [ ]:
sigmoid_similarities = sigmoid_kernel(ncorpus)